In [12]:
import mediapipe as mp
import numpy as np
import cv2 as cv
import os
# import tensorflow as tf
# from model.keypoint_classifier.coral_keypoint_classifier import CoralKeyPointClassifier
# from model import PointHistoryClassifier
from utils import CvFpsCalc
import copy
import itertools
from collections import Counter
from collections import deque
print(os.getcwd())

import csv

model_path='model/keypoint_classifier/keypoint_classifier.tflite'


/home/pumid/Desktop/athome/athome_ws/src/gesture_detection/scripts


In [14]:
#!/home/pumid/Desktop/athome/bin/python3
# -*- coding: utf-8 -*-
import argparse
import time
import numpy as np
import os
from PIL import Image
from pycoral.adapters import classify
from pycoral.adapters import common
from pycoral.utils.dataset import read_label_file
from pycoral.utils.edgetpu import make_interpreter, list_edge_tpus
import os
print(os.getcwd())
class CoralKeyPointClassifier(object):
    def __init__(self, model_path='src/gesture_detection/scripts/model/keypoint_classifier/keypoint_classifier.tflite'):
        self.interpreter = make_interpreter(model_path_or_content=model_path)
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()

    def __call__(
        self,
        landmark_list,
    ):
        input_details_tensor_index = self.input_details[0]['index']
        self.interpreter.set_tensor(
            input_details_tensor_index,
            np.array([landmark_list], dtype=np.float32))
        self.interpreter.invoke()

        output_details_tensor_index = self.output_details[0]['index']

        result = self.interpreter.get_tensor(output_details_tensor_index)

        result_index = np.argmax(np.squeeze(result))

        return result_index

2022-11-01 17:22:44.034607: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-01 17:22:44.185010: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-01 17:22:44.774270: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/pumid/Desktop/athome/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:/usr/local/cuda-11.2/lib64:/home/pumid/Desktop/fra333/lab2_ws/install/scarlet_kinematics_interfaces/lib:/home/pumid/Des

ImportError: generic_type: type "InterpreterWrapper" is already registered!

: 

In [8]:
##### argument #####
cap_device = 0
cap_width = 960
cap_height = 540
use_static_image_mode = False
min_detection_confidence = 0.7
min_tracking_confidence = 0.5

##### setting camera #####
cap = cv.VideoCapture()
cap.set(cv.CAP_PROP_FRAME_WIDTH, cap_width)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, cap_height)

##### mediapipe hand #####
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
        static_image_mode=use_static_image_mode,
        max_num_hands=1,
        min_detection_confidence=min_detection_confidence,
        min_tracking_confidence=min_tracking_confidence,
    )

##### load model #####
keypoint_classifier = CoralKeyPointClassifier(model_path='model/keypoint_classifier/keypoint_classifier.tflite')
keypoint_classifier_labels = ['Open', 'Close', 'Pointer', 'OK']
# point_history_classifier = PointHistoryClassifier()
# point_history_classifier_labels = ['Stop', 'Clockwise', 'Counter Clockwise', 'Move']


##### measure FPS #####
cvFpsCalc = CvFpsCalc(buffer_len=10)

##### declare history point
history_length = 16
point_history = deque(maxlen=history_length)

# Finger gesture history ################################################
finger_gesture_history = deque(maxlen=history_length)

##### measure fps
fps_que = deque(maxlen=50)


# Classifier Function

In [9]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)
        # landmark_z = landmark.z

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point

def pre_process_landmark(landmark_list):
    temp_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temp_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temp_landmark_list[index][0] = temp_landmark_list[index][0] - base_x
        temp_landmark_list[index][1] = temp_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temp_landmark_list = list(
        itertools.chain.from_iterable(temp_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temp_landmark_list)))

    def normalize_(n):
        return n / max_value

    temp_landmark_list = list(map(normalize_, temp_landmark_list))

    return temp_landmark_list

def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv.boundingRect(landmark_array)

    return [x, y, x + w, y + h]

# History Function

In [10]:
def pre_process_point_history(image, point_history):
    image_width, image_height = image.shape[1], image.shape[0]

    temp_point_history = copy.deepcopy(point_history)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, point in enumerate(temp_point_history):
        if index == 0:
            base_x, base_y = point[0], point[1]

        temp_point_history[index][0] = (temp_point_history[index][0] - base_x) / image_width
        temp_point_history[index][1] = (temp_point_history[index][1] - base_y) / image_height

    # Convert to a one-dimensional list
    temp_point_history = list(
        itertools.chain.from_iterable(temp_point_history))

    return temp_point_history
    
def draw_point_history(image, point_history):
    for index, point in enumerate(point_history):
        if point[0] != 0 and point[1] != 0:
            cv.circle(image, (point[0], point[1]), 1 + int(index / 2),
                      (152, 251, 152), 2)

    return image

# Video Loop

In [11]:
##### setting camera #####
cap = cv.VideoCapture(cap_device)
cap.set(cv.CAP_PROP_FRAME_WIDTH, cap_width)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, cap_height)
while cap.isOpened():
    fps = cvFpsCalc.get()
    fps_que.append(fps)

    if cv.waitKey(10) & 0xFF == ord('q'):
        break

    ret, image = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    image = cv.flip(image, 1)
    debug_image = copy.deepcopy(image)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True

    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            ##### test normalize 
            landmark_point = []
            for _, landmark in enumerate(hand_landmarks.landmark):
                landmark_point.append([landmark.x, landmark.y, landmark.z])
            
            ##### mediapipe output normarlize convert to pixel
            landmark_list = calc_landmark_list(debug_image, hand_landmarks)

            ##### hand classifier
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            hand_sign_id = keypoint_classifier(pre_processed_landmark_list)
            print(hand_sign_id)

            # ##### store hand classifier
            # if hand_sign_id == 2:  # Point gesture
            #     point_history.append(landmark_list[8])
            # else:
            #     point_history.append([0, 0])

            # ##### point history classifier
            # finger_gesture_id = 0
            # if len(point_history) == history_length:
            #     pre_processed_point_history_list = pre_process_point_history(debug_image, point_history)
            #     finger_gesture_id = point_history_classifier(pre_processed_point_history_list)

            # # Calculates the gesture IDs in the latest detection
            # finger_gesture_history.append(finger_gesture_id)
            # most_common_fg_id = Counter(finger_gesture_history).most_common()
            # dynamic_sign = point_history_classifier_labels[most_common_fg_id[0][0]]


            ##### display information|
            ### Draw Landmarks
            mp_drawing.draw_landmarks(debug_image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())

            ### draw bounding box 
            brect = calc_bounding_rect(debug_image, hand_landmarks)
            cv.rectangle(debug_image, (brect[0], brect[1]), (brect[2], brect[3]), (0, 0, 0), 1)

            ### draw info text
            cv.rectangle(debug_image, (brect[0], brect[1]), (brect[2], brect[1] - 22),(0, 0, 0), -1)
            hand_sign_text = keypoint_classifier_labels[hand_sign_id]
            info_text = handedness.classification[0].label[0:]
            if hand_sign_text != "":
                info_text = info_text + ':' + hand_sign_text
            cv.putText(debug_image, info_text, (brect[0] + 5, brect[1] - 4), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv.LINE_AA)
            # finger_gesture_text = dynamic_sign
            # if finger_gesture_text != "":
            #     cv.putText(debug_image, "Finger Gesture:" + finger_gesture_text, (10, 60), cv.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 4, cv.LINE_AA)
            #     cv.putText(debug_image, "Finger Gesture:" + finger_gesture_text, (10, 60), cv.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2, cv.LINE_AA)

                
        
    debug_image = draw_point_history(debug_image, point_history)
    cv.putText(debug_image, "FPS:" + str(fps), (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 4, cv.LINE_AA)
    cv.putText(debug_image, "FPS:" + str(fps), (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2, cv.LINE_AA)

    cv.imshow('Raw Webcam Feed', debug_image)        

cap.release()
cv.destroyAllWindows()

1.1149258613586426
1.0916216373443604
1.06431245803833
1.0720385313034058
1.0715556144714355
1.056807518005371
1.0628643035888672
1.0773143768310547
1.065109372138977
1.0554052591323853
1.060792326927185
1.0441566705703735
1.0295602083206177
1.0323363542556763
1.0284719467163086
1.010884404182434
1.0107656717300415
0.9990731477737427
0.9941638112068176
0.9948581457138062
0.9943894743919373
0.9988430142402649
0.9972467422485352
1.0043838024139404
1.0086121559143066
1.0134936571121216
1.0108006000518799
1.0212690830230713
1.029006838798523
1.036027431488037
1.024812936782837
1.029506802558899
1.0147452354431152
1.0128573179244995
1.014697790145874
1.0193201303482056
1.032262921333313
1.0241705179214478
1.026885747909546
1.0251953601837158
1.0286403894424438
1.0312650203704834
1.032004952430725
1.0249900817871094
1.0275676250457764
1.012315034866333
0.9147886633872986
0.881510317325592
0.8653398156166077
0.8559556007385254
0.8510482311248779
0.8471153378486633
0.844573974609375
0.84212994

# Logging Csv

In [ ]:
csv_path = 'model/keypoint_classifier/test_keypoint.csv'
##### setting camera #####
cap = cv.VideoCapture(cap_device)
cap.set(cv.CAP_PROP_FRAME_WIDTH, cap_width)
cap.set(cv.CAP_PROP_FRAME_HEIGHT, cap_height)
while cap.isOpened():
    fps = cvFpsCalc.get()

    if cv.waitKey(10) & 0xFF == ord('q'):
        break

    ret, image = cap.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    image = cv.flip(image, 1)
    debug_image = copy.deepcopy(image)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True

    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
            ##### mediapipe output normarlize convert to pixel
            landmark_list = calc_landmark_list(debug_image, hand_landmarks)

            ##### hand classifier
            pre_processed_landmark_list = pre_process_landmark(landmark_list)
            with open(csv_path, 'a', newline="") as f:
                writer = csv.writer(f)
                writer.writerow([1, *pre_processed_landmark_list])

            ##### display information|
            ### Draw Landmarks
            mp_drawing.draw_landmarks(debug_image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing_styles.get_default_hand_landmarks_style(), mp_drawing_styles.get_default_hand_connections_style())

    cv.imshow('Raw Webcam Feed', debug_image)        

cap.release()
cv.destroyAllWindows()

In [ ]:
from collections import Counter
c = Counter([1, 1, 2, 3, 1, 1, 1, 0])
print(c)